In [2]:
import pandas as pd
import numpy as np
import os
import re
from icecream import ic
from collections import defaultdict
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [5]:
# basedir = '../../../ready_data'
# fname = 'ADNI_AB_TAU_DATA.csv'
# # fname = 'train_vld_test_split_updated/adni_revised_labels.csv'
# data = pd.read_csv(f'{basedir}/{fname}')
# data

,RID,DIAGNOSIS,AGE_MRI,SCANDATE.T1,LONIUID.T1,PTGENDER,PTEDUCAT,PTTLANG,PTETHCAT,PTRACCAT,AMY_SUVR,TRACER.AMY,MTL_SUVR,NEO-T_SUVR,MMSCORE,CDGLOBAL,VSBPSYS,VSBPDIA,VSPULSE,VSRESP,amy_label,tau_label
0,5273,2.0,92.153320,2015-08-26,I506903,1.0,20.0,1.0,2.0,5.0,1.291,FBP,1.57450,1.54075,27.0,0.5,154.0,84.0,68.0,16.0,1,0
1,5242,1.0,69.363450,2015-08-12,I505147,1.0,20.0,1.0,2.0,5.0,0.897,FBP,1.25800,1.44600,29.0,0.0,129.0,79.0,65.0,16.0,0,0
2,5269,1.0,67.085558,2015-09-02,I508480,1.0,16.0,1.0,2.0,5.0,1.018,FBP,1.37975,1.53200,27.0,0.0,136.0,76.0,75.0,14.0,0,0
3,1414,2.0,82.524298,2015-09-09,I510032,1.0,18.0,1.0,2.0,5.0,1.259,FBP,1.81600,1.58075,26.0,0.5,114.0,58.0,95.0,23.0,1,0
4,4862,2.0,79.570157,2015-08-28,I507327,1.0,18.0,1.0,2.0,5.0,1.515,FBP,2.19825,1.91900,24.0,0.5,163.0,80.0,53.0,16.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,6480,2.0,83.088296,2021-09-02,I1488898,2.0,16.0,1.0,2.0,5.0,1.496,FBP,2.81500,2.04925,29.0,0.5,109.0,84.0,65.0,16.0,1,1
1075,6457,1.0,78.721424,2022-07-22,I1607991,1.0,18.0,1.0,2.0,5.0,1.033,FBP,1.47525,1.64350,29.0,0.5,178.0,76.0,65.0,16.0,0,0
1076,6459,1.0,70.132786,2022-07-20,I1607363,2.0,16.0,1.0,2.0,5.0,0.979,FBP,1.25525,1.42525,30.0,0.5,153.0,82.0,67.0,16.0,0,0
1077,2121,2.0,79.132101,2022-01-18,I1535982,2.0,14.0,1.0,2.0,5.0,1.500,FBP,1.64425,1.63625,30.0,0.5,159.0,75.0,76.0,5.0,1,0


In [9]:
basedir = '../../../ready_data'
fname = 'ADNI_ABTAU_MOREFEATURES.csv'
# fname = 'train_vld_test_split_updated/adni_revised_labels.csv'
data = pd.read_csv(f'{basedir}/{fname}')
data

,RID,PHASE,SITEID,DIAGNOSIS,PTGENDER,PTDOBMM,PTDOBYY,PTEDUCAT,PTTLANG,PTETHCAT,PTRACCAT,LONIUID.T1,LONIUID.FLAIR,LONIUID.AMY,T1_DESCRIPTION,FLAIR_DESCRIPTION,TRACER.AMY,SERIES_QUALITY.T1,SERIES_SELECTED.T1,SERIES_QUALITY.FLAIR,SERIES_SELECTED.FLAIR,IMAGE_RESOLUTION.AMY,QC_FLAG.AMY,AMYLOID_STATUS_COMPOSITE_REF,AMY_SUVR,MTL_SUVR,NEO-T_SUVR,MESIAL_TEMPORAL_SUVR,TEMPO_PARIETAL_SUVR,FRONTAL_SUVR,MTL_VOLUME,NEO-T_VOLUME,TEMPO_PARIETAL_VOLUME,FRONTAL_VOLUME,NEOCORTICAL_SUVR,MMDATE,MMYEAR,MMMONTH,MMDAY,MMSEASON,MMHOSPIT,MMFLOOR,MMCITY,MMAREA,MMSTATE,MMWATCH,MMPENCIL,MMREPEAT,MMHAND,MMFOLD,MMONFLR,MMREAD,MMWRITE,MMDRAW,MMSCORE,WORD1,WORD1DL,WORD2,WORD2DL,WORD3,WORD3DL,WORDLIST,WORLDSCORE,CDMEMORY,CDORIENT,CDJUDGE,CDCOMMUN,CDHOME,CDCARE,CDGLOBAL,CDSOB,WEIGHT,HEIGHT,BPSYS,BPDIAS,HRATE,AGE_MRI,MOCATRAI,MOCACUBE,MOCACLOC,MOCACLON,MOCACLOH,MOCANAMI,MOCAREGI,MOCADIGI,MOCALETT,MOCASER7,MOCAREPE,MOCAFLUE,MOCAABST,MOCARECN,MOCARECC,MOCARECR,MOCAORDT,MOCAORMO,MOCAORYR,MOCAORDY,MOCAORPL,MOCAORCT,MOCATOTS,GDSOURCE,NOGDS,SATIS,DROPACT,EMPTY,BORED,SPIRITS,AFRAID,HAPPY,HELPLESS,STAYHOME,MEMPROB,WONDRFUL,WRTHLESS,ENERGY,HOPELESS,BETTER,GDS,NPISOURCE,DEL,DELSEV,HALL,HALLSEV,AGIT,AGITSEV,DEPD,DEPDSEV,ANX,ANXSEV,ELAT,ELATSEV,APA,APASEV,DISN,DISNSEV,IRR,IRRSEV,MOT,MOTSEV,NITE,NITESEV,APP,APPSEV,NPISCORE,NXVISUAL,NXAUDITO,NXTREMOR,NXCONSCI,NXNERVE,NXMOTOR,NXFINGER,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXOTHER,amy_label,tau_label
0,2002,ADNIGO,48,2.0,1.0,7,1945,16.0,1.0,2.0,5.0,I179549,I179557,I1601460,MPRAGE,Axial T2-FLAIR,FBP,1.0,1.0,2.0,1.0,6mm,2,0.0,0.955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,0.5,0.0,0.0,1.0,0.5,2.5,213.000000,-4.0,133.0,90.0,75.0,64.960986,1.0,1.0,1.0,1.0,1.0,3.0,10.0,2.0,1.0,3,2.0,30.0,2.0,3,2,0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,-4.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0,0
1,2010,ADNIGO,1,2.0,2.0,7,1947,20.0,1.0,2.0,6.0,I180310,I180309,I1592972,MPRAGE,Axial T2-FLAIR,FBP,1.0,1.0,2.0,1.0,6mm,2,0.0,1.122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.5,246.600000,-4.0,165.0,66.0,94.0,62.981520,1.0,0.0,1.0,1.0,1.0,3.0,10.0,2.0,0.0,3,2.0,18.0,2.0,3,0,0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,-4.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.0,1.0,0.0,NaN,0.0,NaN,0.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,0
2,2007,ADNIGO,12,2.0,2.0,12,1926,20.0,1.0,1.0,5.0,I180261,I180260,I1595082,Sag IR-SPGR,Axial FLAIR,FBP,1.0,1.0,1.0,1.0,6mm,2,1.0,1.466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,0.5,0.5,0.5,0.0,0.5,2.5,118.000000,-4.0,120.0,60.0,62.0,83.542779,1.0,0.0,1.0,1.0,1.0,3.0,10.0,2.0,1.0,3,2.0,20.0,1.0,0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,-4.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.0,1.0,0.0,NaN,0.0,NaN,0.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1,0
3,2011,ADNIGO,48,2.0,2.0,7,1940,14.0,1.0,2.0,5.0,I183855,I183861,I1601431,MPRAGE,Axial T2-FLAIR,FBP,1.0,1.0,1.0,1.0,6mm,2,0.0,1.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,0.5,0.0,0.0,0.0,0.5,1.5,146.000000,-4.0,147.0,68.0,83.0,70.031485,1.0,1.0,1.0,1.0,1.0,3.0,10.0,2.0,0.0,3,2.0,22.0,2.0,0,3,2,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,-4.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,NaN,0.0,NaN,

In [8]:
# data[['DIAGNOSIS', 'PTGENDER', 'PTTLANG', 'PTETHCAT', 'PTRACCAT', 'amy_label', 'tau_label']] = data[['DIAGNOSIS', 'PTGENDER', 'PTTLANG', 'PTETHCAT', 'PTRACCAT', 'amy_label', 'tau_label']].astype('category')
# data

,RID,DIAGNOSIS,AGE_MRI,SCANDATE.T1,LONIUID.T1,PTGENDER,PTEDUCAT,PTTLANG,PTETHCAT,PTRACCAT,...,MTL_SUVR,NEO-T_SUVR,MMSCORE,CDGLOBAL,VSBPSYS,VSBPDIA,VSPULSE,VSRESP,amy_label,tau_label
0,5273,2.0,92.153320,2015-08-26,I506903,1.0,20.0,1.0,2.0,5.0,...,1.57450,1.54075,27.0,0.5,154.0,84.0,68.0,16.0,1,0
1,5242,1.0,69.363450,2015-08-12,I505147,1.0,20.0,1.0,2.0,5.0,...,1.25800,1.44600,29.0,0.0,129.0,79.0,65.0,16.0,0,0
2,5269,1.0,67.085558,2015-09-02,I508480,1.0,16.0,1.0,2.0,5.0,...,1.37975,1.53200,27.0,0.0,136.0,76.0,75.0,14.0,0,0
3,1414,2.0,82.524298,2015-09-09,I510032,1.0,18.0,1.0,2.0,5.0,...,1.81600,1.58075,26.0,0.5,114.0,58.0,95.0,23.0,1,0
4,4862,2.0,79.570157,2015-08-28,I507327,1.0,18.0,1.0,2.0,5.0,...,2.19825,1.91900,24.0,0.5,163.0,80.0,53.0,16.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,6480,2.0,83.088296,2021-09-02,I1488898,2.0,16.0,1.0,2.0,5.0,...,2.81500,2.04925,29.0,0.5,109.0,84.0,65.0,16.0,1,1
1075,6457,1.0,78.721424,2022-07-22,I1607991,1.0,18.0,1.0,2.0,5.0,...,1.47525,1.64350,29.0,0.5,178.0,76.0,65.0,16.0,0,0
1076,6459,1.0,70.132786,2022-07-20,I1607363,2.0,16.0,1.0,2.0,5.0,...,1.25525,1.42525,30.0,0.5,153.0,82.0,67.0,16.0,0,0
1077,2121,2.0,79.132101,2022-01-18,I1535982,2.0,14.0,1.0,2.0,5.0,...,1.64425,1.63625,30.0,0.5,159.0,75.0,76.0,5.0,1,0


In [ ]:
data[['DIAGNOSIS', 'PTGENDER', 'PTTLANG', 'PTETHCAT', 'PTRACCAT', 'amy_label', 'tau_label']] = data[['DIAGNOSIS', 'PTGENDER', 'PTTLANG', 'PTETHCAT', 'PTRACCAT', 'amy_label', 'tau_label']].astype('category')
data

In [10]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
percent_missing

{'RID': 0.0,
 'PHASE': 0.0,
 'SITEID': 0.0,
 'DIAGNOSIS': 0.0010053619302949062,
 'PTGENDER': 0.0,
 'PTDOBMM': 0.0,
 'PTDOBYY': 0.0,
 'PTEDUCAT': 0.0,
 'PTTLANG': 0.0,
 'PTETHCAT': 0.0,
 'PTRACCAT': 0.0,
 'LONIUID.T1': 0.0,
 'LONIUID.FLAIR': 0.0023458445040214475,
 'LONIUID.AMY': 0.0,
 'T1_DESCRIPTION': 0.0,
 'FLAIR_DESCRIPTION': 0.0020107238605898124,
 'TRACER.AMY': 0.0,
 'SERIES_QUALITY.T1': 0.0,
 'SERIES_SELECTED.T1': 0.0,
 'SERIES_QUALITY.FLAIR': 0.0020107238605898124,
 'SERIES_SELECTED.FLAIR': 0.0020107238605898124,
 'IMAGE_RESOLUTION.AMY': 0.0,
 'QC_FLAG.AMY': 0.0,
 'AMYLOID_STATUS_COMPOSITE_REF': 0.0006702412868632708,
 'AMY_SUVR': 0.0006702412868632708,
 'MTL_SUVR': 0.6310321715817694,
 'NEO-T_SUVR': 0.6310321715817694,
 'MESIAL_TEMPORAL_SUVR': 0.6310321715817694,
 'TEMPO_PARIETAL_SUVR': 0.6310321715817694,
 'FRONTAL_SUVR': 0.6310321715817694,
 'MTL_VOLUME': 0.6310321715817694,
 'NEO-T_VOLUME': 0.6310321715817694,
 'TEMPO_PARIETAL_VOLUME': 0.6310321715817694,
 'FRONTAL_VOLUME':

In [13]:
mydtype = data['DIAGNOSIS'].dtype
if 'category' in str(mydtype):
    print(mydtype)

category


In [14]:
for i, col in enumerate(data.columns):
    if col == 'RID' or col == 'LONIUID.T1' or col == 'SCANDATE.T1' or col == 'TRACER.AMY':
        continue
    # print(col)
    # print(meta1[meta1['name'] == col]['type'].iloc[0])
    meta[col] = {}
    meta[col]['name'] = col
    meta[col]['unique_values'] = list(data[col].dropna().unique())
    meta[col]['num_unique_values'] = len(meta[col]['unique_values'])
    meta[col]['length'] = 1
    mydtype = data[col].dtype
    if 'category' in str(mydtype):
        meta[col]['type'] = 'C'
    else:
        meta[col]['type'] = 'N'
    meta[col]['missing_rate'] = round(percent_missing[col], 2)
# meta['img_MRI_T1'] = {'name': 'img_MRI_T1', 'type': 'M'}

In [15]:
df = pd.DataFrame(meta).T[['name', 'missing_rate', 'type', 'num_unique_values', 'length', 'unique_values']]

In [16]:
df

,name,missing_rate,type,num_unique_values,length,unique_values
DIAGNOSIS,DIAGNOSIS,0.0,C,3,1,"[2.0, 1.0, 3.0]"
AGE_MRI,AGE_MRI,0.0,N,1031,1,"[92.1533196440794, 69.36344969199179, 67.08555..."
PTGENDER,PTGENDER,0.0,C,2,1,"[1.0, 2.0]"
PTEDUCAT,PTEDUCAT,0.0,C,12,1,"[20.0, 16.0, 18.0, 19.0, 14.0, 15.0, 12.0, 8.0..."
PTTLANG,PTTLANG,0.0,C,2,1,"[1.0, 2.0]"
PTETHCAT,PTETHCAT,0.0,C,3,1,"[2.0, 1.0, 3.0]"
PTRACCAT,PTRACCAT,0.0,C,6,1,"[5.0, 6.0, 4.0, 7.0, 2.0, 1.0]"
AMY_SUVR,AMY_SUVR,0.0,N,547,1,"[1.291, 0.897, 1.018, 1.259, 1.515, 0.964, 1.6..."
MTL_SUVR,MTL_SUVR,0.0,N,988,1,"[1.5745, 1.2579999999999998, 1.379749999999999..."
NEO-T_SUVR,NEO-T_SUVR,0.0,N,928,1,"[1.54075, 1.4460000000000002, 1.532, 1.58075, ..."


In [18]:
df.to_csv('../meta_files/meta_file.csv', index=False)